In [2]:
import spacy
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from ml_datasets import imdb
import requests
import csv
train_data, valid_data = imdb()
nlp = spacy.load("en_core_web_sm")

In [20]:
def get_sample_wikipedia():
    url = "https://olivers-things.s3.amazonaws.com/sample-wikipedia.json"
    articles = requests.get(url).json()
    print(f"fetched {len(articles)} articles")
    return articles

def get_sample_wikipedia_untrained():
    exclude = [a[2] for a in get_train_data()]
    return [a for a in get_sample_wikipedia() if a["title"] not in exclude]

def get_train_data():
    url = "https://docs.google.com/spreadsheets/u/1/d/1ysgN8UoVY942gPVToxIEML_I-q27d9zTO6hAxoW5Yx4/export?format=csv&gid=0"
    tags = requests.get(url)
    decoded = tags.content.decode('utf-8')
    results = []
    
    for row in list(csv.reader(decoded.splitlines(), delimiter=','))[1:]:
        try:
            match = [a for a in articles if row[0] == a["title"]][0]
            title = match["title"]
            content = " ".join(match["sentences"])

            if row[1] == "0":
                results.append((content,"neg",title))
            elif row[1] == "1":
                results.append((content,"pos",title))
        except Exception as e:
            print(f"Error with {row[0]}")
            
    return results

def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    print(f"training on {len(data)} docs")
    
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'neg':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0

        docs.append(doc)
    return docs

***
***CREATE MODEL***
***

In [4]:
articles = get_sample_wikipedia()

train_data = get_train_data()

fetched 665 articles
Error with Albert III
Error with Madison County
Error with Oswego County
Error with Fort Collins
Error with Lincoln
Error with Soay
Error with University of California
Error with Celebration
Error with Rockville
Error with John Stuart
Error with Harlingen


In [ ]:
num_texts = 100
# first we need to transform all the training data
train_docs = make_docs(train_data[:num_texts])
print(f"training on {len(train_docs)} documents")
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./data/train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./data/valid.spacy")

***
***TEST MODEL***
***

In [6]:
nlp = spacy.load("output/model-best")

In [23]:
untrained = get_sample_wikipedia_untrained()

Error with Albert III
Error with Madison County
Error with Oswego County
Error with Fort Collins
Error with Lincoln
Error with Soay
Error with University of California
Error with Celebration
Error with Rockville
Error with John Stuart
Error with Harlingen
fetched 665 articles


In [34]:
def wikipedia_link(title):
    return f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"

for a in untrained[0:50]:
    doc = nlp(" ".join(a["sentences"]))
    result = 1 if doc.cats['positive'] > 0.5 else 0
    print(wikipedia_link(a["title"]), result)

https://en.wikipedia.org/wiki/Albert_III,_Duke_of_Saxony 0
https://en.wikipedia.org/wiki/Book_of_Hosea 1
https://en.wikipedia.org/wiki/Biotite 1
https://en.wikipedia.org/wiki/Casino 0
https://en.wikipedia.org/wiki/Celtic_music 0
https://en.wikipedia.org/wiki/Common_descent 1
https://en.wikipedia.org/wiki/Bubble_Bobble 0
https://en.wikipedia.org/wiki/Constantinople 0
https://en.wikipedia.org/wiki/California 0
https://en.wikipedia.org/wiki/Community_college 1
https://en.wikipedia.org/wiki/Casino_game 1
https://en.wikipedia.org/wiki/Curium 1
https://en.wikipedia.org/wiki/Politics_of_the_Czech_Republic 0
https://en.wikipedia.org/wiki/Chicano 0
https://en.wikipedia.org/wiki/Carabiner 1
https://en.wikipedia.org/wiki/Cardiff 0
https://en.wikipedia.org/wiki/Carlo_Goldoni 0
https://en.wikipedia.org/wiki/Ceuta 0
https://en.wikipedia.org/wiki/Chicago_Bears 0
https://en.wikipedia.org/wiki/Constantin_von_Tischendorf 0
https://en.wikipedia.org/wiki/Cetus 0
https://en.wikipedia.org/wiki/Carnivore 1
h